In [1]:
import os
import json
from glob import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime, timedelta

from google.cloud import bigquery
from google.oauth2 import service_account

pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#matplotlib 패키지 한글 깨짐 처리 시작
plt.rc('font', family='AppleGothic') #맥

import warnings
warnings.filterwarnings(action='ignore')

# main data
df = pd.read_csv('ga4_data.csv')

# cluster data
clustering = pd.read_csv('clustering.csv')

In [2]:
# 전처리 
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'])
df['totals_totalTransactionRevenue'] = df['totals_totalTransactionRevenue'].fillna(0)
df['hits_transaction_transactionRevenue'] = df['hits_transaction_transactionRevenue'].fillna(0)
df.visitStartTime = (df.visitStartTime - timedelta(hours=8))

# 매번 필요
def pad_with_zeros(string):
    """
    문자열의 길이가 19이 될 때까지 앞에 0을 추가하는 함수
    """
    # 문자열의 길이를 세기
    length = len(string)
    
    # 문자열의 길이가 20이 될 때까지 앞에 0을 추가
    while length < 19:
        string = '0' + string
        length += 1
    
    return string

clustering['fullVisitorId'] = clustering['fullVisitorId'].astype(str)
clustering['fullVisitorId'] = clustering['fullVisitorId'].apply(pad_with_zeros)

df['fullVisitorId'] = df['fullVisitorId'].astype(str)
df['fullVisitorId'] = df['fullVisitorId'].apply(pad_with_zeros)

# merge
main = df.merge(clustering, on='fullVisitorId', how='left')

In [3]:
zero = main[main['labels'] == 0]
total_revenue = zero['hits_transaction_transactionRevenue'].sum()
channel_revenue = zero.groupby(['channelGrouping'])['hits_transaction_transactionRevenue'].sum()
channel_revenue = round(channel_revenue / total_revenue * 100, 2) 
channel_revenue.sort_values(ascending=False)

channelGrouping
Referral          47.71
Direct            25.90
Organic Search    21.19
Paid Search        3.26
Display            1.50
Social             0.38
Affiliates         0.05
(Other)            0.00
Name: hits_transaction_transactionRevenue, dtype: float64

In [4]:
channel_revenue.to_csv('acquisition.csv', index=False)